# Agent DQN avec reprise d'expérience et estimation de la fonction d'action cible fixée

Dans ce carnet Jupyter, nous avons implémenté un agent DQN avec une mémoire d'expérience et une cible fixe. Il est entrainé pour résoudre l'environnement "Banana" de Unity. Nous avons utilisé la bibliothèque PyTorch pour construire le réseau de neurones et gérer l'entraînement. L'agent DQN utilise une mémoire d'expérience pour stocker les transitions et un réseau de neurones pour approximer la fonction Q. Une cible fixe est mise à jour périodiquement pour stabiliser l'apprentissage.

Les contraintes de l'environnement sont les suivantes :
- L'agent doit collecter des bananes jaunes tout en évitant les bananes bleues.
- L'agent reçoit une récompense positive pour chaque banane jaune collectée et une récompense négative pour chaque banane bleue collectée.
- L'agent doit apprendre à naviguer dans l'environnement pour maximiser sa récompense totale.
- L'agent dispose de 300 pas de temps pour collecter le maximum de bananes jaunes, ensuite l'épisode d'interactions est indiqué comme terminé par l'environnement.

Un épisode est considéré comme réussi si l'agent collecte au moins 13 bananes.

L'objectif est d'atteindre une moyenne glissante sur 100 épisodes de score supérieure ou égale à 13. Un autre défi (optionnel) consiste à résoudre l'environnement "Banana" en moins de 1800 épisodes. Dans ce carnet, nous avons ajouté une autre contrainte en considérant qu'il fallait que la moyenne glissante sur 100 épisodes soit conservée sur plusieurs épisodes pour considérer l'atteinte de l'objectif comme stable. Par ailleurs, nous avons ajouté la possibilité d'indiquer des contraintes plus fortes sur le nombre maximum de pas de temps disponibles durant un épisode, le nombre d'épisodes pour résoudre l'environnement.

Les définitions des composantes de l'agent DQN (classe `DQNAgentExpReplay`) sont incluses dans des fichiers séparés pour une meilleure lisibilité et modularité, et appelées dans ce carnet.

Le carnet est divisé en plusieurs sections :
1. **Importation des bibliothèques** : Nous importons les bibliothèques nécessaires pour l'entraînement de l'agent DQN.
2. **Initialisation des paramètres et définitions des fonctions globales** : nous définitions les paramètres globaux et les fonctions utiles dans ce carnet.
3. **Entraînement de l'agent** : Nous définissons et utilisons la fonction `train_agent` qui entraîne l'agent DQN sur l'environnement "Banana". Cette étape peut être passée si l'agent a déjà été entraîné, disposant alors d'un fichier des poids du modèle sous-jacent, et que nous souhaitons seulement évaluer ses performances.
4. **Évaluation de l'agent** : Nous évaluons les performances de l'agent sur l'environnement "Banana".


> Note : Certaines parties de ce carnet et les codes inclus sont en partie inspirés des codes fournis par Udacity dans le cadre du projet de la formation "Deep Reinforcement Learning Nanodegree". Nous avons également utilisé des ressources en ligne pour nous aider à construire l'agent DQN et à gérer l'entraînement. Nous avons veillé à respecter les bonnes pratiques de codage et à commenter le code pour faciliter la compréhension.

## Chargement des modules

Importation des bibliothèques nécessaires

In [ ]:
# Built-in and 3rd party modules
from typing import Callable
from collections import namedtuple
import random
import os

import numpy as np
from unityagents import UnityEnvironment
import matplotlib.pyplot as plt
%matplotlib inline
import torch
from tqdm.notebook import trange

# Custom modules
from dqn_agent__expreplay_fixedqtarget import DQNAgentExpReplayFixedQTarget as DQNAgent

## Definitions et initialisations

Nous définissons un typage particulier pour les statistiques des épisodes.

In [ ]:
Episode_Stats = namedtuple("Experience", field_names=["i_episode", "steps_to_resolution", "score", "is_solution"])

Nous définissons une fonction d'affichage des courbes des statistiques et des scores obtenus durant l'apprentissage ou l'évaluation.

In [ ]:
def plot_training_stats(
        episode_stats: list[Episode_Stats],
        mean_scores: list[float],
        shifted_score_avgs: list[float],
        solution_threshold: int,
        scores_window_length: int,
        title: str="") -> None:
    """Display the training statistics in several plots.
    
    Args:
        episode_stats (list[Episode_Stats]): List of episode statistics.
        mean_scores (list[float]): List of mean scores.
        shifted_score_avgs (list[float]): List of shifted score averages.
        solution_threshold (int): Threshold for a solution.
        scores_window_length (int): Length of the window score average.
        title (str): Title for the plot.
    """
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(111)

    scores = [episode.score for episode in episode_stats]
    ax.plot(np.arange(1, len(scores)+1), scores, color='b', label='Scores')
    # Plot the mean scores
    ax.plot(
        np.arange(1, len(mean_scores)+1),
        mean_scores,
        color='y',
        linestyle='--',
        label='Score average from start'
        )
    # Plot the shifted score averages
    ax.plot(
        np.arange(scores_window_length, scores_window_length + len(shifted_score_avgs)),
        shifted_score_avgs,
        color='r',
        label=f"{scores_window_length} episodes shifted average"
        )
    ax.set_ylabel('Score')
    
    # Plot the episode length (count of steps) when successful
    x_scatter = []
    y_scatter = []
    max_steps_to_resolution = 0
    for episode in episode_stats:
        if episode.is_solution:
            x_scatter.append(episode.i_episode)
            y_scatter.append(episode.steps_to_resolution)
            max_steps_to_resolution = max(max_steps_to_resolution, episode.steps_to_resolution)
    ax2 = ax.twinx()
    ax2.scatter(
        x_scatter,
        y_scatter,
        s=2,
        color='g',
        hatch="x",
        label='Steps to success'
    )
    ax2.set_ylim(0, int(1.1 * max_steps_to_resolution))
    ax2.set_ylabel('Steps')

    # Plot the solution threshold line
    ax.axhline(y=solution_threshold, color='k', linestyle='--', label='Solution threshold')
    
    # Display the figure with legend
    ax.set_xlabel('Episode #')
    plt.title(title)
    fig.legend()
    plt.show(loc="lower right")

Nous définissons une fonction permettant de lancer un environnement Unity.

In [ ]:
def launch_unity_env(file_name: str, train_mode: bool = False) -> UnityEnvironment:
    """Launch the Unity environment.
    
    Args:
        file_name (str): Path to the Unity environment executable.
        train_mode (bool): Whether to launch in training mode or not.
    """
    env = UnityEnvironment(file_name)

    # Get the default brain
    brain_name = env.brain_names[0]
    brain = env.brains[brain_name]

    env_info = env.reset(train_mode=train_mode)[brain_name]

    # Number of agents in the environment
    print('Number of agents:', len(env_info.agents))
    
    return env, brain_name, brain

### Initialisations

#### Device

Nous allons utiliser un GPU si disponible, sinon nous utiliserons le CPU. `mps` est le GPU d'Apple.

In [ ]:
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
    )
print(f"Availabe device: {device}")

#### Paramètres globaux

Nous utilisons si nécessaire une graine aléatoire pour la reproductibilité des résultats.

In [ ]:
SEED = 71 # Set the random seed for reproducibility

if SEED is not None:
    random.seed(SEED)
    torch.manual_seed(SEED)
    np.random.seed(SEED)

Nous définissons les hyperparamètres utiles à la définition de l'agent, son entraînement et son évaluation. 

Par ailleurs, nous renforçons les contraintes sur le nombre maximum de pas de temps disponibles durant un épisode avec `max_timesteps`, sur le nombre maximum d'épisodes pour résoudre l'environnement avec `n_episodes_train`, et ajoutons la contrainte de stabilité pour considérer l'atteinte de l'objectif comme stable avec `window_stability`.

In [ ]:
# Agent neural netwotk settings
model_parameters = {
    "fc1_units": 64,
    "fc2_units": 64,
}

# Training hyperparameters
## Epsilon scheduler parameters
eps_end = 0.01
eps_decay = 0.995

n_episodes_train = 1500         # Maximum number of training episodes to solve the environment
max_timesteps = 250             # Max number of timesteps per episode. After 300 timestep the episode is done, as observed in the "Take Random Actions in the Environment" test
window_stability = 5            # Number of episodes to consider the agent learning as stable
avg_window_length_scores = 100  # Project success constraint
solution_threshold = 13         # Project success constraint
print_stats_each_n_episode = 25

weight_filename_prefix = "model_weights"

# Evaluation hyperparameters
n_episodes_test = 500

Nous appellons et initialisons l'environnement Unity, en reprenant les indications fournies dans le carnet `Navigation.ipynb`, et nous récupérons les informations utile pour les interactions avec l'environnement. Cette initialisation peut servir dans ce carnet à la fois pour l'entraînement et l'évaluation de l'agent.

In [ ]:
env, brain_name, brain = launch_unity_env("Banana.app", train_mode=True)

## Entrainement de l'agent

Définition de la fonction d'entrainement d'un agent DQN sur un environnement Unity.

In [ ]:
def train_dqn_agent(
        agent: DQNAgent,
        env: UnityEnvironment,
        brain_name: str,
        eps_scheduler: callable,
        eps_start: float = 1.0,
        n_max_episodes: int = 2000,
        solution_threshold: int = 13,
        avg_window_length_scores: int = 100,
        print_stats_each_n_episode: int = 25,
        window_stability: int = 1,
        max_timesteps: int = 300,
        weight_filename_prefix: str = "checkpoint", # None to not save weights
        ) -> tuple[list[Episode_Stats], list[float], list[float]]:
    """Train the DQN agent in the environment.
    
    Args:
        agent (DQNAgentExpReplay): The DQN agent to train.
        env (UnityEnvironment): The Unity environment.
        brain_name (str): The name of the brain in the environment.
        eps_scheduler (function): Function to schedule epsilon.
        eps_start (float): Initial epsilon value.
        n_max_episodes (int): Number of episodes to train the agent.
        solution_threshold (int): Threshold for a solution.
        avg_window_length_scores (int): Length of the window score average.
        print_stats_each_n_episode (int): Frequency of printing stats.
        window_stability (int): Window length for stability check.
        max_timesteps (int): Maximum number of timesteps per episode.
        weight_filename_prefix (str): Prefix for saving weights. If None, weights are not saved. Filename will be suffixed with "_solved" or "_last_episode", the weights are saved when the environment is solved or at the end of training, respectively.

    Returns:
        tuple: A tuple containing:
            - stats_episodes (list[Episode_Stats]): List of episode statistics.
            - score_avgs (list[float]): List of mean scores.
            - shift_score_avgs (list[float]): List of shifted score averages.
    """
    # List of stats for each episode
    scores = []
    score_avgs = []
    shifted_score_avgs = []
    stats_episodes = []

    eps = eps_start             # Epsilon for exploration
    solved = False              # Flag to ind6icate if the environment is solved in the episode
    last_solved = 0             # Last episode indice where the environment was solved
    solved_episode_count = 0    # Count of episodes where the environment was solved

    agent.train()
    for i_episode in trange(1, n_max_episodes+1, desc="Training", unit="episode", leave=False):
        # Reset the environment
        env_info = env.reset(train_mode=True)[brain_name]
        # Initial state
        state = env_info.vector_observations[0]

        score = 0               # episode score
        actions_of_episode = [] # list of actions taken during the episode
        steps_to_resolution = 0
        # Loop for each episode
        for i_step in range(1, max_timesteps+1):
            # Agent chooses action
            action = agent.act(state, eps=eps)
            actions_of_episode.append(action)
            
            # Apply action to environment and get environment evolution as
            # experience : next state, reward and done
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0] 
            
            score += reward
            if score >= solution_threshold and steps_to_resolution == 0:
                steps_to_resolution = i_step
            
            # Agent learns from the experience
            agent.step(state, action, reward, next_state, done)

            # Move to the next state
            state = next_state                
            
            # Check if episode is done (a.k.a terminal state)
            if done:
                break 
        
        # Save episode stats
        stats_episodes.append(
            Episode_Stats(i_episode, steps_to_resolution, score, score >= solution_threshold)
            )
        scores.append(score)
        score_avgs.append(np.mean(scores))
        
        if score >= solution_threshold:
            last_solved = i_episode
            solved_episode_count += 1

        # Process shifted score average when enough scores
        if i_episode >= avg_window_length_scores:
            shifted_score_avgs.append(np.mean(scores[-avg_window_length_scores:]))

            # Print stats regularly
            if i_episode % print_stats_each_n_episode == 0:
                print(f'\rEpisode {i_episode} | Mean scores {score_avgs[-1]:.2f}' + \
                        f'| {avg_window_length_scores} shift score average: {shifted_score_avgs[-1]:.2f}' + \
                        f'| Environment solved {solved_episode_count} time(s).',
                        end=" ")
                if last_solved > 0:
                    print(f'Last solution at episode {last_solved}th with score {stats_episodes[-1].score}.', end="")

            # Check the stability of the solution
            if np.mean(shifted_score_avgs[-window_stability:]) >= solution_threshold:
                print(f'\n>> Environment solved in {i_episode} episodes!\tAverage Score: {shifted_score_avgs[-1]:.2f} stable')
                if weight_filename_prefix is not None:
                    filename = f"{weight_filename_prefix}_solved.pth"
                    agent.save(filename)
                    print(f"Model weights saved to {filename}")
                solved = True
                break

        # Update epsilon
        eps = eps_scheduler(i_step, eps)

    # Save the last episode if environment not solved
    if not solved:
        print(f'\n>> Environment not solved after {i_episode} episodes!\tAverage Score: {shifted_score_avgs[-1]:.2f} (perhaps > {solution_threshold} but not stable yet)')
        if weight_filename_prefix is not None:
            filename = f"{weight_filename_prefix}_last_episode.pth"
            agent.save(filename)
            print(f"\nModel weights saved to {filename}")

    return stats_episodes, score_avgs, shifted_score_avgs

Nous créons un agent en indiquant les paramètres d'initialisation, et nous l'entraînons sur l'environnement "Banana" en utilisant la fonction `train_agent`. A noter la création de `eps_scheduler` qui doit respecter une certaine interface (deux paramètres d'entrée `i_episode` et `eps`) pour être compatible avec la fonction d'entraînement de l'agent.

In [ ]:
# Instanciate the agent
agent = DQNAgent (
    state_size=brain.vector_observation_space_size,
    action_size=brain.vector_action_space_size,
    model_parameters=model_parameters,
    device=device
    )

# eps_scheduler : function to modify epsilon
# i_episode (based on 1) is the number of steps
# eps is the current epsilon
eps_scheduler = lambda i_episode, eps, : max(eps_end, eps_decay*eps)

# Train the agent
stats_episodes, score_avgs, shifted_score_avgs = train_dqn_agent(
    agent,
    env,
    brain_name,
    eps_scheduler,
    eps_start=1.0,
    n_max_episodes=n_episodes_train,
    avg_window_length_scores=avg_window_length_scores,
    solution_threshold=solution_threshold,
    window_stability=window_stability,
    print_stats_each_n_episode=print_stats_each_n_episode,
    weight_filename_prefix=weight_filename_prefix
)

Nous affichons les courbes des stastistiques et des scores obtenus durant l'apprentissage.

In [ ]:
plot_training_stats(
    stats_episodes, score_avgs, shifted_score_avgs, solution_threshold, avg_window_length_scores,
    "Training : scores and score averages"
    )

## Evaluation de l'agent

Nous évaluons les performances de l'agent sur l'environnement "Banana" en utilisant la fonction. Nous affichons les courbes d'apprentissage et les scores obtenus durant le test.

L'évaluation peut porter sur l'agent qui vient d'être entrainé, ou sur un agent dont les poids sont disponibles dans un fichier.

In [ ]:
weight_filename = "model_weights_solved.pth" # Prefix for saving weights
reload = True # Set to True to reload the last training weights
if reload:
    agent = DQNAgent(
        state_size=brain.vector_observation_space_size,
        action_size=brain.vector_action_space_size,
        model_parameters=model_parameters,
        device=device
        )
    agent.load(weight_filename)
agent.eval()

Cette étape permet de forcer l'affichage de la fenêtre de l'environnement Unity, afin de préparer les éventuels enregistrements d'écran pour des vidéos de démonstration.

In [ ]:
if env is None:
    env, brain_name, brain = launch_unity_env("Banana.app", train_mode=False)
else:
    env_info = env.reset(train_mode=False)[brain_name]

Nous définissons une fonctions d'évaluation de l'agent sur un environnement Unity.

In [ ]:
def eval_dqn_agent(
        agent: DQNAgent,
        env: UnityEnvironment,
        brain_name: str,
        n_episodes_test: int = 100,
        solution_threshold: int = 13,
        avg_window_length_scores: int = 100,
        window_stability: int = 1
    ) -> tuple[list[Episode_Stats], list[float], list[float]]:
    """Evaluate the DQN agent in the environment.
    
    Args:
        agent (DQNAgent): The DQN agent to evaluate.
        env (UnityEnvironment): The Unity environment.
        brain_name (str): The name of the brain in the environment.
        n_episodes_test (int): Number of episodes to test the agent.
        solution_threshold (int): Threshold for a solution.
        avg_window_length_scores (int): Length of the window score average.
        window_stability (int): Window length for stability check.

    Returns:
        tuple: A tuple containing:
            - stats_episodes (list[Episode_Stats]): List of episode statistics.
            - score_avgs (list[float]): List of mean scores.
            - shift_score_avgs (list[float]): List of shifted score averages.
    """
    scores = []
    stats_episodes = []
    score_avgs = []
    shifted_score_avgs = []

    score_avg = 0
    for i_episode in trange(1, n_episodes_test+1):
        env_info = env.reset(train_mode=False)[brain_name]
        state = env_info.vector_observations[0]
        
        score = 0    
        done = False
        steps_to_resolution = 0
        while not done:
            # Agent chooses action
            action = agent.act(state)
            
            # Apply action to environment
            env_info = env.step(action)[brain_name]
            
            # Get environment evolution as experience : next state, reward and done
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0]
            state = next_state

            # Update the score
            score += reward                               
            steps_to_resolution += 1

            print(f"\rEpisode #{i_episode} : Score = {int(score)} in {steps_to_resolution} steps| Score avg = {score_avg:.2f}", end="")

        # Save episode stats
        stats_episodes.append(
                Episode_Stats(i_episode, steps_to_resolution, score, score >= solution_threshold)
                )
        scores.append(score)
        score_avg = np.mean(scores)
        score_avgs.append(score_avg)
        
        # Process shifted score average when enough scores
        if i_episode >= avg_window_length_scores:
            shifted_score_avgs.append(np.mean(scores[-avg_window_length_scores:]))
            # Check the shifted score average stability
            if np.mean(shifted_score_avgs[-window_stability:]) >= solution_threshold:
                print(f"\rEpisode #{i_episode} : Score = {int(score)} in {steps_to_resolution} steps| Score avg = {score_avgs[-1]:.2f}", end="")
                break

    return stats_episodes, score_avgs, shifted_score_avgs

Nous lançons l'évaluation de l'agent sur l'environnement "Banana".

In [ ]:
eval_stats_episodes, eval_score_avgs, eval_shifted_score_avgs = eval_dqn_agent(
    agent,
    env,
    brain_name,
    n_episodes_test=n_episodes_test,
    solution_threshold=solution_threshold,
    avg_window_length_scores=avg_window_length_scores,
    window_stability=window_stability
)

Puis nous affichons les courbes de scores obtenus durant l'évaluation, constatant que les objectifs fixés ont été atteints, même avec une contrainte plus forte sur le nombre maximum de pas de temps disponibles durant un épisode et la stabilité de la moyenne glissante sur 100 épisodes.

In [ ]:
plot_training_stats(
    eval_stats_episodes, eval_score_avgs, eval_shifted_score_avgs, solution_threshold, avg_window_length_scores,
    "Testing : scores and score averages"
    )

## Fin du carnet

Nous fermons l'environnement Unity ouvert.

In [ ]:
env.close()